In [1]:
import pandas as pd
import numpy as np

In [2]:
import glob
import os
import re
import random
import pandas as pd
import re
import nltk
import string

In [3]:
from ast import literal_eval as load
def NER_format(file, shift = 0):
    df = pd.read_csv(file)
    words = []
    sentence_id = []
    labels = []
    pos_tag = []
    for i in range(len(df)):
        word = load(df.iloc[i,4])
        BIO = load(df.iloc[i,5])
        pos = load(df.iloc[i,6])
        sentence_id.extend([i+shift]*len(word))
        words.extend(word)
        labels.extend(BIO)
        pos_tag.extend(pos)
    return sentence_id, words, labels, pos_tag

In [4]:
sentence_id, words, labels, pos_tag = NER_format("Dataset_TaskB/Training_Phrases.csv")
df1 = pd.DataFrame({"sentence_id": sentence_id, "words": words, "labels": labels, "pos_tag" : pos_tag})
df1.dropna(subset = ['words', 'labels'], inplace = True)
df1.to_csv("Dataset_TaskB/NCG_Train.csv", index = None)

In [5]:
sentence_id, words, labels, pos_tag = NER_format("Dataset_TaskB/Trial_Phrases.csv")
df2 = pd.DataFrame({"sentence_id": sentence_id, "words": words, "labels": labels, "pos_tag" : pos_tag})
df2.dropna(subset = ['words', 'labels'], inplace = True)
df2.to_csv("Dataset_TaskB/NCG_Valid.csv", index = None)

In [24]:
def tag(x):
    tokens = x.split()
    tags = nltk.pos_tag(tokens)
    tags = [x[1] for x in tags]
    return tags

In [25]:
path = "sciERC_raw/raw_data/*.txt"
files = glob.glob(path)
sentence_id, sentence, list_words, BIO, pos_tag, length = [], [], [], [], [], []
for file1 in files:
    file2 = file1[0:-4] + ".ann"
    with open(file1, encoding='latin-1') as f:
        lines = f.readlines()
    sent = lines[-1].strip()
    with open(file2, encoding='latin-1') as f:
        lines = f.readlines()
    start_index = []
    end_index = []
    for line in lines:
        item = line.split()
        if(len(item) < 5 or item[0][0] != 'T'):
            continue
        start_index.append(int(item[2]))
        end_index.append(int(item[3]))
    start_index.sort()
    end_index.sort()
    words = sent.split()
    sent_id = file1[20:-4]
    cur, index = 0, 0
    labels = []
    for i, word in enumerate(words):
        if(index<len(start_index) and cur > end_index[index]):
            index += 1
        if(index >= len(start_index)):
            labels.append('O')
            continue
        if(cur >= start_index[index] and i==0):
            labels.append('B')
            continue
        if(cur >= start_index[index] and i>0 and labels[i-1] == 'O'):
            labels.append('B')
        elif (cur >= start_index[index]):
            labels.append('I')
        else:
            labels.append('O')
        cur = cur + 1 + len(word)
    pos = tag(sent)
    sentence_id.append(sent_id)
    sentence.append(sent)
    list_words.append(words)
    BIO.append(labels)
    pos_tag.append(pos)
    length.append(len(words))
df_phrase = pd.DataFrame({'sentence_id': sentence_id, 'sentence' : sentence, 'words': list_words, 
                          'BIO' : BIO, 'POS' : pos_tag, 'length' : length})

In [26]:
df_phrase

,sentence_id,sentence,words,BIO,POS,length
0,N03-1004,Motivated by the success of ensemble methods ...,"[Motivated, by, the, success, of, ensemble, me...","[O, O, O, O, O, O, B, I, O, B, I, O, O, O, O, ...","[VBN, IN, DT, NN, IN, JJ, NNS, IN, NN, NN, CC,...",124
1,INTERSPEECH_2006_28_abs,We investigated whether automatic phonetic tra...,"[We, investigated, whether, automatic, phoneti...","[O, O, O, B, I, I, I, O, O, B, I, I, I, O, O, ...","[PRP, VBD, IN, JJ, JJ, NNS, VBP, MD, VB, RB, V...",136
2,ICCV_1999_47_abs,Background maintenance is a frequent element o...,"[Background, maintenance, is, a, frequent, ele...","[B, I, I, I, I, O, O, O, O, B, I, I, O, B, I, ...","[NNP, NN, VBZ, DT, JJ, NN, IN, NN, NN, NN, PRP...",104
3,ICCV_2011_47_abs,We present a method for detecting 3D objects u...,"[We, present, a, method, for, detecting, 3D, o...","[O, O, O, B, O, B, I, I, O, B, O, B, O, O, O, ...","[PRP, VBP, DT, NN, IN, VBG, CD, NNS, VBG, NN, ...",86
4,CVPR_2016_416_abs,We investigate the problem of fine-grained ske...,"[We, investigate, the, problem, of, fine-grain...","[O, O, O, O, O, B, I, I, I, I, O, B, I, I, O, ...","[PRP, VBP, DT, NN, IN, JJ, JJ, NN, NN, NNP, WR...",176
...,...,...,...,...,...,...
495,H93-1076,Two themes have evolved in speech and text im...,"[Two, themes, have, evolved, in, speech, and, ...","[O, O, B, O, O, O, B, I, I, I, I, O, O, O, O, ...","[CD, NNS, VBP, VBN, IN, NN, CC, JJ, NN, NN, NN...",126
496,CVPR_1996_15_abs,We previously presented a framework for segmen...,"[We, previously, presented, a, framework, for,...","[O, O, O, O, B, O, B, I, I, I, O, O, B, I, O, ...","[PRP, RB, VBD, DT, NN, IN, NN, IN, JJ, NNS, VB...",117
497,ICASSP_2011_588_abs,In a motorized vehicle a number of easily meas...,"[In, a, motorized, vehicle, a, number, of, eas...","[O, O, O, O, O, O, O, O, B, I, O, B, I, O, O, ...","[IN, DT, JJ, NN, DT, NN, IN, RB, JJ, NNS, IN, ...",146
498,H90-1011,This paper describes a particular approach to ...,"[This, paper, describes, a, particular, approa...","[O, O, O, O, O, O, B, O, B, O, O, O, O, O, B, ...","[DT, NN, VBZ, DT, JJ, NN, TO, VBG, IN, JJ, JJ,...",147


In [27]:
df_phrase.to_csv("Dataset_TaskB/SCIERC_Paragraphs.csv", index = None)

In [28]:
df_phrase = pd.read_csv("Dataset_TaskB/SCIERC_Paragraphs.csv")

In [29]:
from ast import literal_eval as load
def NER_format(df):
    shift = 0
    words = []
    sentence_id = []
    labels = []
    pos_tag = []
    for i in range(len(df)):
        word = load(df.iloc[i,2])
        BIO = load(df.iloc[i,3])
        pos = load(df.iloc[i,4])
        sentence_id.extend([i+shift]*len(word))
        words.extend(word)
        labels.extend(BIO)
        pos_tag.extend(pos)
    return sentence_id, words, labels, pos_tag

In [30]:
sentence_id, words, labels, pos_tag = NER_format(df_phrase)

In [31]:
df = pd.DataFrame({"sentence_id": sentence_id, "words": words, "labels": labels, "pos_tag" : pos_tag})

In [33]:
df = df[df['sentence_id'] < 496]

In [34]:
df.to_csv("Dataset_TaskB/scierc_paragraphs_phrases.csv", index = None)

In [35]:
df = pd.read_csv("Dataset_TaskB/scierc_paragraphs_phrases.csv")
df.dropna(subset = ['words', 'labels'], inplace = True)
df.to_csv("Dataset_TaskB/scierc_paragraphs_phrases.csv", index = None)

In [36]:
df_phrase['length'].mean()

121.46

In [37]:
from ast import literal_eval as load
def NER_format(df):
    words = []
    sentence_id = []
    labels = []
    pos_tag = []
    index = 0
    for i in range(len(df)):
        word = load(df.iloc[i,2])
        BIO = load(df.iloc[i,3])
        pos = load(df.iloc[i,4])
        words.extend(word)
        labels.extend(BIO)
        pos_tag.extend(pos)
    for word in words:
        sentence_id.append(index)
        if(word.endswith('.')):
            index += 1
    print(index)
    return sentence_id, words, labels, pos_tag

In [38]:
sentence_id, words, labels, pos_tag = NER_format(df_phrase)

2715


In [39]:
df = pd.DataFrame({"sentence_id": sentence_id, "words": words, "labels": labels, "pos_tag" : pos_tag})

In [40]:
df[df['labels'] == 'B']

,sentence_id,words,labels,pos_tag
6,0,methods,B,NNS
9,0,learning,B,NN
15,0,language,B,NN
22,0,and,B,CC
38,0,agents,B,NNS
...,...,...,...,...
60700,2713,near-Bernoulli,B,JJ
60704,2713,highly,B,RB
60708,2714,approach,B,NN
60712,2714,real,B,JJ


In [41]:
df = df[df['sentence_id'] < 2704]

In [42]:
df.to_csv("Dataset_TaskB/scierc_sent_phrases.csv", index = None)

In [43]:
df = pd.read_csv("Dataset_TaskB/scierc_sent_phrases.csv")

In [44]:
df.dropna(subset = ['words', 'labels'], inplace = True)

In [45]:
df.to_csv("Dataset_TaskB/scierc_sent_phrases.csv", index = None)

In [46]:
file = open('TDMSci.txt', 'r')
lines = file.readlines()
sentence_id = []
words = []
labels = []
sent_id = 0
for line in lines:
    tokens = line.split('\t')
    if(len(tokens) < 3):
        sent_id += 1
    else:
        sentence_id.append(sent_id)
        words.append(tokens[0])
        labels.append(tokens[2][0])

In [47]:
sent_id

2009

In [48]:
df = pd.DataFrame({"sentence_id": sentence_id, "words": words, "labels": labels})

In [49]:
df = df[df['sentence_id'] < 2000]

In [50]:
df.to_csv("Dataset_TaskB/TDMSci_dataset.csv", index = None)

In [51]:
def tag(tokens):
    tags = nltk.pos_tag(tokens)
    tags = [x[1] for x in tags]
    return tags

In [52]:
def process_data_TDMSci():
    df = pd.read_csv("Dataset_TaskB/TDMSci_dataset.csv")
    df.dropna(subset = ['words', 'labels'], inplace = True)
    sentences = df.groupby("sentence_id")["words"].apply(list).values
    pos_tag = []
    for sent in sentences:
        tags = tag(sent)
        pos_tag.extend(tags)
    df['pos_tag'] = pos_tag
    df.to_csv("Dataset_TaskB/TDMSci_dataset.csv", index = None)
process_data_TDMSci()

In [53]:
df = pd.read_csv("Dataset_TaskB/TDMSci_dataset.csv")
df.dropna(subset = ['words', 'labels'], inplace = True)
df.to_csv("Dataset_TaskB/TDMSci_dataset.csv", index = None)